# XCH4 dataset satellite lev2

## Import libraries

In [ ]:
import cartopy.crs as ccrs
import flox.xarray
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot

plt.style.use("seaborn-v0_8-notebook")

## Define request

In [ ]:
# Time parameters
year_start = 2005
year_stop = 2006

collection_id = "satellite-methane"
request = {
    "processing_level": "level_2",
    "variable": "xch4",
    "sensor_and_algorithm": "merged_emma",
    "version": "4.4",
    "year": [
        str(year) for year in range(year_start - 1, year_stop + 1)
    ],  # Download previous year to include Dec
    "month": [f"{i:02d}" for i in range(1, 13)],
    "day": [f"{i:02d}" for i in range(1, 32)],
}

## Define function to cache

In [ ]:
def arithmetic_unweighted_average(ds, d_lon, d_lat, lon1):
    if lon1 not in (180, 360):
        raise ValueError(f"lon1 must be 180 or 360. {lon1=}")
    lon0 = -180 if lon1 == 180 else 0

    coords = {}
    expected_groups = ()
    for name, start, stop, step in zip(
        ["latitude", "longitude"], [-90, lon0], [90, lon1], [d_lat, d_lon]
    ):
        coords[name] = np.arange(start + step / 2, stop + step / 2, step)
        groups = np.arange(start, stop + step, step)
        groups[0] -= step
        expected_groups += (pd.IntervalIndex.from_breaks(groups),)

    ds = flox.xarray.xarray_reduce(
        ds, *coords, func="mean", expected_groups=expected_groups, keep_attrs=True
    )
    ds = ds.rename({f"{coord}_bins": coord for coord in coords}).assign_coords(coords)
    for coord in ds.coords:
        ds[coord].attrs["standard_name"] = coord
    return ds


def monthly_regrid(ds, d_lon, d_lat, lon1=180):
    """Resample to monthly regular grid.

    Parameters
    ----------
    ds: Dataset
        Dataset to resample
    d_lon, d_lat: int
        Longitude/latitude step size
    lon1: int, {180, 360}
        Right longitude bound. According to which convention is used,
        longitudes will vary from -180 to 180 or from 0 to 360.

    Returns
    -------
    Dataset
    """
    ds = ds.set_coords(["longitude", "latitude"])
    ds_out = ds.resample(time="1MS").map(
        arithmetic_unweighted_average, d_lon=d_lon, d_lat=d_lat, lon1=lon1
    )
    return ds_out

## Download and transform data

In [ ]:
ds = download.download_and_transform(
    collection_id,
    request,
    chunks={"year": 1},
    transform_func=monthly_regrid,
    transform_func_kwargs={"d_lon": 1, "d_lat": 1, "lon1": 180},
)

## Compute seasonal anomalies

In [ ]:
# Select years (shift -1 to get D(year-1)J(year)F(year))
ds = ds.assign_coords(year=ds["time"].dt.year.shift(time=-1).astype(int))
mask = (ds["year"] >= year_start) & (ds["year"] <= year_stop)
ds = ds.where(mask.compute(), drop=True)

# Compute anomaly
climatology = diagnostics.seasonal_weighted_mean(ds)
with xr.set_options(keep_attrs=True):
    anomaly = ds.groupby("year").map(diagnostics.seasonal_weighted_mean) - climatology

# Update attributes
for da in anomaly.data_vars.values():
    if "long_name" in da.attrs:
        da.attrs["long_name"] += " Anomaly"

## Plot seasonal anomalies

In [ ]:
_ = plot.projected_map(
    anomaly["xch4"], projection=ccrs.Robinson(), col="season", row="year", robust=True
)